In [7]:
from langchain.prompts.chat import ChatPromptTemplate

In [8]:
from langserve import RemoteRunnable

openai_llm = RemoteRunnable("http://localhost:8000/openai/")

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a highly educated person who loves to use big words. "
            + "You are also concise. Never answer in more than three sentences." 
        ),
        ("human", "Tell me about your favorite novel")
    ]
).format_messages()

In [11]:
openai_llm.invoke(prompt)

AIMessage(content='My favorite novel is "Pride and Prejudice" by Jane Austen. It is a witty and insightful commentary on society, filled with complex characters and social satire. Austen\'s writing style is elegant and engaging.', response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 43, 'total_tokens': 88}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d457d4ac-694f-4891-aa1e-5ca15c844f0f-0', usage_metadata={'input_tokens': 43, 'output_tokens': 45, 'total_tokens': 88})

In [14]:
mistral = RemoteRunnable("http://localhost:8000/mistral/")

mistral.invoke(prompt)

' My favorite novel is Victor Hugo\'s "Les Misérables," an intricately woven tapestry of second chances and redemption, set against the backdrop of 19th-century France. The narrative explores themes of love, sacrifice, and social injustice through its captivating characters, ultimately culminating in a profound exploration of the human spirit.'

# Chaining two chains 

In [12]:
from langchain.schema.runnable import RunnablePassthrough

comedian_chain = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a comedian that sometimes tells funny jokes and other times you just state facts that are not funny. Please either tell a joke or state fact now but only output one.",
            ),
        ]
    )
    | openai_llm
)

joke_classifier_chain = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Please determine if the joke is funny. Say `funny` if it's funny and `not funny` if not funny. Then repeat the first five words of the joke for reference...",
            ),
            ("human", "{joke}"),
        ]
    )
    | openai_llm
)


chain = {"joke": comedian_chain} | RunnablePassthrough.assign(
    classification=joke_classifier_chain
)


In [13]:
chain.invoke({})

{'joke': AIMessage(content='Did you hear about the mathematician who’s afraid of negative numbers? He’ll stop at nothing to avoid them!', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 42, 'total_tokens': 65}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7b28ede8-28b5-42e5-b0b0-f8ea47b5b37c-0', usage_metadata={'input_tokens': 42, 'output_tokens': 23, 'total_tokens': 65}),
 'classification': AIMessage(content='not funny\nDid you hear about the...', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 196, 'total_tokens': 205}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ab36f660-5e70-4f34-a07a-925707f375b1-0', usage_metadata={'input_tokens': 196, 'output_tokens': 9, 'total_tokens': 205})}